## Framingham Risk Calculator - Jupyter

INSTRUCTIONS (<i>from the the MDCalc version of the calculator found [here](https://www.mdcalc.com/framingham-risk-score-hard-coronary-heart-disease)</i>):

There are several distinct Framingham risk models. MDCalc uses the 'Hard' coronary Framingham outcomes model, which is intended for use in non-diabetic patients age 30-79 years with no prior history of coronary heart disease or intermittent claudication, as it is the most widely applicable to patients without previous cardiac events. See the official Framingham [website](https://www.framinghamheartstudy.org/fhs-risk-functions/hard-coronary-heart-disease-10-year-risk/) for additional Framingham risk models.

### Overview

This notebook will include the following:
* We will use Python to program our own version of the Framingham Risk Calculator. 
* We will pull FHIR resources from our sandboxes to test our calculator.

A final version of the calculator is included in its own cell at the end of the notebook for those who don't want to go through the entire notebook.

### Adding Framingham patients to your sandbox

We will be using the same FHIR sandbox that we used in the query activity. Framingham requires specific data points (blood pressure, cholesterol levels, medications, and smoking status) that none of the sample patients have. 

Logica sandboxes have a data importing portal that we will use to add three new patients that have the data we need. 

1. Download the <i>fram-resource-bundle.json</i> file from the Project folder onto your computer.
![download-file](imgs/download-file.png)
2. Select the "Data Manager" tab on the left sidebar of the sandbox.
3. Select "IMPORT".
4. Click the "LOAD FROM FILE" button in the top right corner.
5. Select the <i>fram-resource-bundle.json</i> file you downloaded in step 1.
![add-bundle](imgs/add-bundle.png)

You should see a list of "201 created" responses in the "RESULTS" tab. 

![201-created](imgs/201-created.png)

You can use the "Patient" tab on the left sidebar of the sandbox to confirm that there are three new patients with associated data. The identifiers will differ between each sandbox, but keep them at hand because they will be used in the queries below.

![new-patient](imgs/new-patients.png)

### Calculating the risk

The formula for both men and women can be found [here](https://www.mdcalc.com/framingham-risk-score-hard-coronary-heart-disease#evidence). We have translated it into python functions, one for men and one for women (since the calculation varies slightly by gender).

In [2]:
def male_fram(age,smoker,total_chol,hdl_chol,sys,treated):
    if age < 30 or age > 79:
        return ['old']
    if age > 70:
        age_smoke = 70
    else:
        age_smoke = age
            
    l = 52.000961*math.log(age) + 20.014077*math.log(total_chol) + -0.905964*math.log(hdl_chol) + \
    1.305784*math.log(sys) + 0.241549*treated + 12.096316*smoker + -4.605038*math.log(age)*math.log(total_chol) + \
    -2.84367*math.log(age_smoke)*smoker + -2.93323*math.log(age)*math.log(age) - 172.300168
    
    prob = 1 - 0.9402**math.exp(l)
    if age > 75:
        return [prob*100,'old']
    else:
        for group in male_avg:
            if age in group:
                return [prob*100, male_avg[group]]

In [3]:
def female_fram(age,smoker,total_chol,hdl_chol,sys,treated):
    if age < 30 or age > 79:
        return ['old']
    if age > 78:
        age_smoke = 78
    else:
        age_smoke = age
            
    l = 31.764001*math.log(age) + 22.465206*math.log(total_chol) + -1.187731*math.log(hdl_chol) + \
    2.552905*math.log(sys) + 0.420251*treated + 13.07543*smoker + -5.060998*math.log(age)*math.log(total_chol) + \
    -2.996945*math.log(age_smoke)*smoker - 146.5933061
    
    prob = 1 - 0.98767**math.exp(l)
    if age > 75:
        return [prob*100,'old']
    else:
        for group in female_avg:
            if age in group:
                return [prob*100, female_avg[group]]

We also need reference ranges to determine whether a risk score is normal or high for a given age. We have included two dictionaries that will give reference information for certain ages by gender.

In [1]:
import math
male_avg = {range(30,35):'1%',range(35,45):'4%',range(45,50):'8%',range(50,55):'10%',range(55,60):'13%',range(60,65):'20%',range(65,70): '22%',range(70,75):'25%'}
female_avg = {range(30,35):'<1%',range(35,40):'<1%',range(40-45):'1%',range(45,50):'2%',range(50,55):'3%',range(55,60):'7%',range(60,65):'8%',range(65,70): '8%',range(70,75):'11%'}

### Pulling FHIR data

We need the following data points for each patient in order to calculate risk:
* Age
* Gender
* Smoking status (Yes = 1, No = 0)
* Systolic blood pressure
* Total cholesterol
* HDL cholesterol
* Is the patient's blood pressure being treated with medicines? (Yes = 1, No = 0)

We will use the Python requests library for executing our queries. This means we need the appropriate URL for each of the resources we want. A helpful tool for building future request URLs is the clinFHIR [Server Query](http://clinfhir.com/query.html) tool. It will help you see which request fields are available for each resource. The alternative would be to find request example URLs in the FHIR specification for each resource.

Our sandboxes are our data servers, you will need the same open data URL you used in the query activity.

In [51]:
import requests,json,datetime,pandas
base = 'replace-this-with-your-open-url'

#### Patient resource - name, age, and gender

Typically, a FHIR app is launched within the EHR and, as a result, will have the ID of the patient resource for the subject.

In [42]:
r = requests.get(base + '/Patient?_id=fake')

A query to a FHIR server will typically return a bundle resource. If it did not find the resource you were asking for then the bundle won't contain any resources and will look something like this.

In [43]:
print(r.text)

{
  "resourceType": "Bundle",
  "id": "402f5f0e-4102-40b9-bb85-2ba6ee787a43",
  "meta": {
    "lastUpdated": "2020-02-14T19:03:18.896+00:00"
  },
  "type": "searchset",
  "total": 0,
  "link": [
    {
      "relation": "self",
      "url": "https://api.logicahealth.org/fhirlabreportsandbox/open/Patient?_id=fake"
    }
  ]
}


FHIR resources are in the JSON format. This means that all data fields have a key and a value. However, be aware that some values can be lists of additional key:value pairs. You can identify lists because they use square brackets [ ]. We will print out the result of a valid query so you can get an idea for what this looks like.

To query a patient, we will be using their IDs. Refer back to your "Patient" view in the sandbox to find the patient IDs for your Framingham patients.

In [52]:
patient_id = 'replace-this-with-your-target-patient-id'
r = requests.get(base + '/Patient?_id=' + patient_id)
print(r.text)

{
  "resourceType": "Bundle",
  "id": "e93d8c92-42d4-43ef-9190-ee3b1304ad14",
  "meta": {
    "lastUpdated": "2020-02-24T21:04:13.851+00:00"
  },
  "type": "searchset",
  "total": 1,
  "link": [
    {
      "relation": "self",
      "url": "https://api.logicahealth.org/framinghamrisk/open/Patient?_id=16702"
    }
  ],
  "entry": [
    {
      "fullUrl": "https://api.logicahealth.org/framinghamrisk/open/Patient/16702",
      "resource": {
        "resourceType": "Patient",
        "id": "16702",
        "meta": {
          "versionId": "1",
          "lastUpdated": "2020-02-24T13:58:19.000+00:00"
        },
        "text": {
          "status": "generated",
          "div": "<div xmlns=\"http://www.w3.org/1999/xhtml\"><div class=\"hapiHeaderText\">Mr. Dmitri <b>1FRAM-LOW </b></div><table class=\"hapiPropertyTable\"><tbody><tr><td>Date of birth</td><td><span>19 June 1982</span></td></tr></tbody></table></div>"
        },
        "name": [
          {
            "use": "official",
      

Remember, if the value of a key is a list, you must specify which item in the list you want. As an example, we will extract the actual patient resource from the bundle resource which is the default return type for server queries. Since this is the only patient resource with the ID we used, there is only one item in the "entry" list. This means we want to take the item at position 0 of the "entry" list (the 1st item). 

Python currently sees our response as a string. Before we can extract fields we must cast it as a JSON object using the json.loads() command. 

In [53]:
bundle = json.loads(r.text)
p = bundle['entry'][0]['resource']
print(p)

{'resourceType': 'Patient', 'id': '16702', 'meta': {'versionId': '1', 'lastUpdated': '2020-02-24T13:58:19.000+00:00'}, 'text': {'status': 'generated', 'div': '<div xmlns="http://www.w3.org/1999/xhtml"><div class="hapiHeaderText">Mr. Dmitri <b>1FRAM-LOW </b></div><table class="hapiPropertyTable"><tbody><tr><td>Date of birth</td><td><span>19 June 1982</span></td></tr></tbody></table></div>'}, 'name': [{'use': 'official', 'family': '1Fram-Low', 'given': ['Dmitri'], 'prefix': ['Mr.']}], 'gender': 'male', 'birthDate': '1982-06-19'}


We can extract the gender of the patient using a similar command

In [54]:
gender = p['gender']
print(gender)

male


The 'name' field can be confusing at first. The 'given' field (first name) is usually a list while the 'family' field (last name) is usually just a string.

In [55]:
first = p['name'][0]['given'][0]
last = p['name'][0]['family']
name = first + ' ' + last
print(name)

Dmitri 1Fram-Low


Age will also present some difficulty since we are only provided with the date of birth. Thankfully, Python's datetime library has some helpful tools for working with dates. We can extract the birth year by splitting the string on '-' and selecting the first item in the resulting list. Then, if we convert that to an integer, we can calculate the age.

In [56]:
birth_year = p['birthDate'].split('-')[0]
current_year = datetime.date.today().year
age = current_year - int(birth_year)
print(age)

38


#### Observation resource - smoking status, systolic BP, total cholesterol, HDL cholesterol

Observation resources are a very generic resource and can represent many different things. To make sure that everyone takes a standardized approach to using observations, profiles have been created to constrain how to use observations to represent clinical concepts.

A good first step when building the query for a concept you haven't worked with before is to consult the US Core Profile [guide](https://build.fhir.org/ig/HL7/US-Core-R4/). The guidelines change depending on the FHIR version you are using. This example uses the R4 version of FHIR.

If we look into the guide we'll see that there is a defined SmokingStatus [profile](https://build.fhir.org/ig/HL7/US-Core-R4/StructureDefinition-us-core-smokingstatus.html). As we explore the profile we find that a SmokingStatus observation should be given the LOINC code 72166-2. We'll also see that the possible values for each SmokingStatus observation are constrained to a ValueSet of the possible statuses: Current every day smoker, Current some day smoker, Former smoker, Never smoker, Smoker - current status unknown, Unknown if ever smoked, Current Heavy tobacco smoker, Current Light tobacco smoker.

We can then query for the most recent observation with a 'subject' equal to our target patient ID and which also has a 'code' of 72166-2.

If the SmokingStatus observation is 'Never Smoker' then we will give the patient a 'No' value for the calculator, otherwise, we will assume that they are a smoker and will give a 'Yes' value.

In [57]:
url = base + '/Observation?subject=' + patient_id + '&_sort:desc=date&code=72166-2'
smoker = 1

#Send the query
r = requests.get(url)
#Create a JSON object of the result
bundle = json.loads(r.text)
#Check to make sure a resource was returned
if bundle['total'] != 0:
    #Parse the contents of that resource
    status = bundle['entry'][0]['resource']['valueCodeableConcept']['text']
    if status == 'Never smoker':
        smoker = 0
        print(status)
    else:
        print(status)
else:
    print('None found')

Never smoker


Blood pressure is part of the Vital Signs Panel [profile](http://hl7.org/fhir/R4/observation-vitalsigns.html). From that profile we see that there is a defined structure for representing blood pressure levels. We will query for the most recent blood pressure observation and will extract the systolic measurement which are stored as a component in that observation.

In [58]:
url = base + '/Observation?subject=' + patient_id + '&_sort:desc=date&code=55284-4'
sys = ''
#Send the query
r = requests.get(url)
#Create a JSON object of the result
bundle = json.loads(r.text)
#Check to make sure a resource was returned
if bundle['total'] != 0:
    #Parse the contents of that resource
    for component in bundle['entry'][0]['resource']['component']:
        #Check for the systolic value (we don't want dialstolic)
        if component['code']['text'] == 'Systolic Blood Pressure':
            sys = component['valueQuantity']['value']
            print(sys)
else:
    print('None found')

125


Cholesterol values are stored together as part of a single lipid panel, as described in the lipid lab report [profile](https://www.hl7.org/fhir/lipidprofile.html). The actual results are stored as Observation resources. We will search for the most recent values.

In [59]:
#Total cholesterol
url = base + '/Observation?subject=' + patient_id + '&_sort:desc=date&code=2093-3'
total_chol = ''
#Send the query
r = requests.get(url)
#Create a JSON object of the result
bundle = json.loads(r.text)
#Check to make sure a resource was returned
if bundle['total'] != 0:
    #Parse the contents of that resource
    total_chol = bundle['entry'][0]['resource']['valueQuantity']['value']
    print(total_chol)
else:
    print('None found')

200


In [60]:
#HDL cholesterol
url = base + '/Observation?subject=' + patient_id + '&_sort:desc=date&code=2085-9'
hdl_chol = ''
#Send the query
r = requests.get(url)
#Create a JSON object of the result
bundle = json.loads(r.text)
#Check to make sure a resource was returned
if bundle['total'] != 0:
    #Parse the contents of that resource
    hdl_chol = bundle['entry'][0]['resource']['valueQuantity']['value']
    print(hdl_chol)
else:
    print('None found')

58


#### MedicationRequest resource - Is the patient's blood pressure being treated with medicines? 

This last piece of needed information will give a good introduction to working with patient medications. There are a few resources which each represent different aspects of how medications are used in a clinical setting. They are each described in the [R4 Medications Module](http://hl7.org/fhir/medications-module.html). A MedicationRequest resource is generated when the provider creates the instruction for a patient to take a certain medication. As you can imagine, this is usually the most convenient way to see which medications a patient should be taking or should have taken at some point of time. Whether or not the patient actually takes the medication is another story. We will assume that if a MedicationRequest resource exists for a certain medication that the patient took that medication as prescribed.

We will also need to determine whether a given medication is a being used to treat blood pressure or not. To do this we will use a ValueSet resource. Explained [here](https://www.hl7.org/fhir/valueset.html), these are handy resources for constraining possible values which, for our case, is blood pressure medications. A ValueSet resource was created for this example and is stored in the server. It contains 5 values for possible blood pressure medications which are common to our synthetic patients. We will query the ValueSet resource by title which is "blood-pressure-medication" and will extract the associated medication codes. We will then build our URL from this ValueSet to search for any MedicationRequest resources with codes in that URL.

In [61]:
#ValueSet
url = base + '/MedicationRequest?subject=' + patient_id + '&_sort:desc=date&code='
r = requests.get(base + '/ValueSet?title=blood-pressure-medications')
bundle = json.loads(r.text)
valueset = bundle['entry'][0]['resource']['expansion']['contains']
for meds in valueset:
    #Add each code to the URL we will use for our MedicationRequest query
    url += meds['code'] + ','
#Take out trailing comma
url = url[:-1]

#MedicationRequest
treated = 0
#Send the query
r = requests.get(url)
#Create a JSON object of the result
bundle = json.loads(r.text)
#If any resources were returned, then they have been treated
if bundle['total'] > 0:
    treated = 1
    print(treated)
else:
    print('None found')

None found


### Putting it all together

The last step is to send each of the variables which we have collected through FHIR resources to the calculating functions. To do this, we will create functions for each of the processes described above (so that you don't need to run each cell in this notebook sequentially) and will call the appropriate calculating function. 

In [62]:
import requests,json,datetime,pandas,math

base = 'replace-this-with-your-open-url'
patient_id = 'replace-this-with-your-target-patient-id'

male_avg = {range(30,35):'1%',range(35,45):'4%',range(45,50):'8%',range(50,55):'10%',range(55,60):'13%',range(60,65):'20%',range(65,70): '22%',range(70,75):'25%'}
female_avg = {range(30,35):'<1%',range(35,40):'<1%',range(40-45):'1%',range(45,50):'2%',range(50,55):'3%',range(55,60):'7%',range(60,65):'8%',range(65,70): '8%',range(70,75):'11%'}

def launch():    
    #DEMOGRAPHICS
    r = requests.get(base + '/Patient?_id=' + patient_id)
    bundle = json.loads(r.text)
    demographics = parseDemographics(bundle);
    if demographics[0] == 'error':
        print('---ERROR---\n' + 'Patient not found: ' + patient_id)
        
    #SMOKING STATUS
    r = requests.get(base + '/Observation?subject=' + patient_id + '&_sort:desc=date&code=72166-2')
    bundle = json.loads(r.text)
    smoker = parseSmoker(bundle)
    
    #SYSTOLIC BLOOD PRESSURE
    r = requests.get(base + '/Observation?subject=' + patient_id + '&_sort:desc=date&code=55284-4')
    bundle = json.loads(r.text)
    sys = parseSystolic(bundle)
    if sys == 'error':
        print('---ERROR---\n' + 'No systolic blood pressure values for patient: ' + patient_id)
        
    #TOTAL CHOLESTEROL
    r = requests.get(base + '/Observation?subject=' + patient_id + '&_sort:desc=date&code=2093-3')
    bundle = json.loads(r.text)
    total_chol = parseCholesterol(bundle)
    if total_chol == 'error':
        print('---ERROR---\n' + 'No total cholesterol values for patient: ' + patient_id)
        
    #HDL CHOLESTEROL
    r = requests.get(base + '/Observation?subject=' + patient_id + '&_sort:desc=date&code=2085-9')
    bundle = json.loads(r.text)
    hdl_chol = parseCholesterol(bundle);
    if hdl_chol == 'error':
        print('---ERROR---\n' + 'No HDL cholesterol reading for patient: ' + patient_id)
        
    #TREATED WITH BLOOD PRESSURE MEDICATIONS?
    url = base + '/MedicationRequest?subject=' + patient_id + '&_sort:desc=date&code='
    #First, get ValueSet for BP medications
    r = requests.get(base + '/ValueSet?title=blood-pressure-medications')
    bundle = json.loads(r.text)
    for meds in bundle['entry'][0]['resource']['expansion']['contains']:
        #Add each code to the URL we will use for our MedicationRequest query
        url += meds['code'] + ',';
    #Take out trailing comma
    url = url[:-1];
    #Query for MedicationRequest resources for the patient
    r = requests.get(url)
    bundle = json.loads(r.text)
    treated = parseMeds(bundle);
    
    #CALCULATE RISK
    risk = getRisk(demographics,smoker,sys,total_chol,hdl_chol,treated)
    result = compileReport(demographics,smoker,sys,total_chol,hdl_chol,treated,risk)
    print(result)

def parseDemographics(bundle): 
    try:
        p = bundle['entry'][0]['resource']
        #Name
        first = p['name'][0]['given'][0]
        last = p['name'][0]['family']
        name = first + ' ' + last
        #Gender
        gender = p['gender']
        #Age
        birth_year = p['birthDate'].split('-')[0]
        current_year = datetime.date.today().year
        age = current_year - int(birth_year)
        return [name,gender,age]
    except:
        return ['error']

def parseSmoker(bundle):
    #0 = no smoking history, 1 = any smoking history
    if bundle['total'] != 0: 
        smoker = 1
        if bundle['entry'][0]['resource']['valueCodeableConcept']['text'] == 'Never smoker':
            smoker = 0
        return smoker
    else:
        return 0 #Default to no if nothing is found

def parseSystolic(bundle):
    if bundle['total'] != 0:
        for comp in bundle['entry'][0]['resource']['component']:
            if comp['code']['text'] == 'Systolic Blood Pressure': 
                return comp['valueQuantity']['value']
        return 'error'
    else:
        return 'error'
    
def parseCholesterol(bundle):
    if bundle['total'] != 0:
        return bundle['entry'][0]['resource']['valueQuantity']['value']
    else:
        return 'error'


def parseMeds(bundle):
    treated = 0; #Default treated status is 0 (No)
    #If any entries are returned then we know they are blood pressure medications
    if bundle['total'] != 0:
        treated = 1
    return treated;

def getRisk(demographics,smoker,sys,total_chol,hdl_chol,treated):
    gender = demographics[1]
    age = demographics[2]
    if gender == 'female':
        r = female_fram(age,smoker,sys,total_chol,hdl_chol,treated)
    else:
        r = male_fram(age,smoker,sys,total_chol,hdl_chol,treated)
    risk = r[0];
    #If the patient is 80 or older, the calculation wasn't performed
    if risk == 'old':
        return 'Risk can only be calculated for ages 30-79'
    else:
        avg = r[1];
        #If the patient is above 74 years old there is no 'Average' information to include in the result
        if avg == 'old':
            return '10-year risk of MI or death: ' + str(round(risk, 2)) + '% (No average reference above 74 years old)'
        else:
            return '10-year risk of MI or death: ' + str(round(risk, 2)) + '%  (Average: ' + avg + ')';

def compileReport(demographics,smoker,sys,total_chol,hdl_chol,treated,risk):
    report = ''
    name = demographics[0]
    report += 'Name:\t\t\t\t\t' + name + '\n'
    gender = demographics[1];
    report += 'Gender:\t\t\t\t\t' + gender + '\n'
    age = demographics[2];
    report += 'Age:\t\t\t\t\t' + str(age) + '\n'
    if smoker == 1:
        report += 'Smoker:\t\t\t\t\tYES\n'
    else:
        report += 'Smoker:\t\t\t\t\tNO\n'
    report += 'Systolic BP:\t\t\t\t' + str(sys) + '\n'
    report += 'Total Cholesterol:\t\t\t' + str(total_chol) + '\n'
    report += 'HDL Cholesterol:\t\t\t' + str(hdl_chol) + '\n'
    if treated == 1:
        report += 'Blood Pressure treated by medication:\tYES\n'
    else:
        report += 'Blood Pressure treated by medication:\tNO\n'
    report += '--------RESULT--------\n'
    report += risk
    return report

def male_fram(age,smoker,sys,total_chol,hdl_chol,treated):
    if age < 30 or age > 79:
        return ['old']
    if age > 70:
        age_smoke = 70
    else:
        age_smoke = age
            
    l = 52.000961*math.log(age) + 20.014077*math.log(total_chol) + -0.905964*math.log(hdl_chol) + \
    1.305784*math.log(sys) + 0.241549*treated + 12.096316*smoker + -4.605038*math.log(age)*math.log(total_chol) + \
    -2.84367*math.log(age_smoke)*smoker + -2.93323*math.log(age)*math.log(age) - 172.300168
    
    prob = 1 - 0.9402**math.exp(l)
    if age > 75:
        return [prob*100,'old']
    else:
        for group in male_avg:
            if age in group:
                return [prob*100, male_avg[group]]
            
def female_fram(age,smoker,sys,total_chol,hdl_chol,treated):
    if age < 30 or age > 79:
        return ['old']
    if age > 78:
        age_smoke = 78
    else:
        age_smoke = age
            
    l = 31.764001*math.log(age) + 22.465206*math.log(total_chol) + -1.187731*math.log(hdl_chol) + \
    2.552905*math.log(sys) + 0.420251*treated + 13.07543*smoker + -5.060998*math.log(age)*math.log(total_chol) + \
    -2.996945*math.log(age_smoke)*smoker - 146.5933061
    
    prob = 1 - 0.98767**math.exp(l)
    if age > 75:
        return [prob*100,'old']
    else:
        for group in female_avg:
            if age in group:
                return [prob*100, female_avg[group]]
            
launch()

Name:					Dmitri 1Fram-Low
Gender:					male
Age:					38
Smoker:					NO
Systolic BP:				125
Total Cholesterol:			200
HDL Cholesterol:			58
Blood Pressure treated by medication:	NO
--------RESULT--------
10-year risk of MI or death: 0.8%  (Average: 4%)
